In [ ]:
!jt -r

In [ ]:
import os, types
import pandas as pd
from botocore.client import Config
#import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_db3aab955d234e4090c6889e4becd5ab = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='F5PYK6UZ84B9Q51VbRJr4NM1hsB3NOrLVytQbOMtdQ1J',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

body = client_db3aab955d234e4090c6889e4becd5ab.get_object(Bucket='drugclassification-donotdelete-pr-qelzilii9u99my',Key='drug200.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

data = pd.read_csv(body)
data.head()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import warnings
import pickle
from scipy import stats
warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')

In [ ]:
# Reading the csv data
df = pd.read_csv('F:\MY PROJECTS\KJCMT COLLEGE PROJECTS\DRUG CLASSIFICATION\data\drug200.csv')
df.head()

In [ ]:
# Shape of csv data
df.shape

In [ ]:
# Finding null values
df.isnull().sum()

In [ ]:
# Checking the information of features
df.info()

# Descriptive analysis

In [ ]:
df.describe(include='all')

# Univariate analysis

In [ ]:
# Checking the distribution (normal or skewed)

plt.figure(figsize=(12,5))
plt.subplot(121)
sns.distplot(df['Age'],color='r')
plt.subplot(122)
sns.distplot(df['Na_to_K'])
plt.show()

In [ ]:
# From the above plot age column is normally distributed. Na_to_k is right skewed (mean>mode). To overcome skewness transformation techniques can be used.

print(stats.mode(df['Na_to_K']))
print(np.mean(df['Na_to_K']))

In [ ]:
# Finding outliers

plt.figure(figsize=(12,5))
plt.subplot(121)
sns.boxplot(df['Age'])
plt.subplot(122)
sns.boxplot(df['Na_to_K'])
plt.show()

In [ ]:
# Na_to_K has 8 outliers. In this project we are not going to handle outliers. Most of the classification algorithms are not sensitive to outliers.

q1 = np.quantile(df['Na_to_K'],0.25)
q3 = np.quantile(df['Na_to_K'],0.75)

IQR = q3-q1

upper_bound = q3+(1.5*IQR)

print('Upper Bound :',upper_bound)

print('Skewed data :',len(df[df['Na_to_K']>upper_bound]))

In [ ]:
# Creating a data frame with categorical features for following visualization

df_cat = df.select_dtypes(include='object')
df_cat.head()

In [ ]:
# Visualizing the count of categorical variable.

plt.figure(figsize=(18,4))
for i,j in enumerate(df_cat):
    plt.subplot(1,4,i+1)
    sns.countplot(df[j])   

# Bivariate analysis

In [ ]:
df.head()

In [ ]:
# Visualizing the relation between drug, BP, sex & cholesterol

plt.figure(figsize=(20,5))
plt.subplot(131)
sns.countplot(df['Drug'],hue=df['BP'])
plt.legend(loc='upper right')
plt.subplot(132)
sns.countplot(df['Drug'],hue=df['Sex'])
plt.subplot(133)
sns.countplot(df['Drug'],hue=df['Cholesterol'])

In [ ]:
# Creating a new column Age_. This column shows the categorized age.

df['Age_'] = ['15-30' if x<=30 else '30-50' if x>30 and x<=50 else '50-75' for x in df['Age']]
df.head()

In [ ]:
# Finding the relation between categorized age and drug

pd.crosstab(df['Age_'],[df['Drug']])

In [ ]:
# Removing the Age_ column

df.drop('Age_',axis=1,inplace=True)
df.head()

# Multivariate Analysis

In [ ]:
sns.swarmplot(df['Drug'],df['Na_to_K'],hue=df['BP'])

# DrugC is used for low BP patient, DrugY is used on patients having Na_to_K > 15.

# Encoding- Converting categorical to numerical

In [ ]:
# Replacing low, normal & high with 0, 1 & 2...

df['BP'] = [0 if x=='LOW' else 1 if x=='NORMAL' else 2 for x in df['BP']]

In [ ]:
# Replacing normal and high cholesterol with 0 & 1

df['Cholesterol'] = [0 if x=='NORMAL' else 1 for x in df['Cholesterol']]

In [ ]:
# Replacing female and male with 0 & 1

df['Sex'] = [0 if x=='F' else 1 for x in df['Sex']]

In [ ]:
df.head()

In [ ]:
df['Drug'].value_counts()

In [ ]:
x = df.drop('Drug',axis=1)
x.head()

In [ ]:
y = df['Drug']
y.head()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=10)

In [ ]:
print('Shape of x_train {}'.format(x_train.shape))
print('Shape of y_train {}'.format(y_train.shape))
print('Shape of x_test {}'.format(x_test.shape))
print('Shape of y_test {}'.format(y_test.shape))

In [ ]:
rf = RandomForestClassifier()
rf.fit(x_train,y_train)

In [ ]:
ypred = rf.predict(x_test)

In [ ]:
confusion_matrix(y_test,ypred)

In [ ]:
print(classification_report(y_test,ypred))

In [ ]:
pickle.dump(rf,open('drugclassification.pkl','wb'))

In [ ]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": "_uQJaQRra2V-2lHNbQY1-q314HMfQ3Rkw1ZhkFeVDtWN"
}
client = APIClient(wml_credentials)

In [ ]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    #print(space)
    return(next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [ ]:
space_uid = guid_from_space_name(client, space_name='Drug Classification')
print("Space UID = " + space_uid)

In [ ]:
client.set.default_space(space_uid)

In [ ]:
client.software_specifications.list()

In [ ]:
import sklearn
sklearn.__version__

In [ ]:
software_spec_uid= client.software_specifications.get_uid_by_name("runtime-22.1-py3.9")
software_spec_uid

In [ ]:
model_details = client.repository.store_model(model='drug-classification-model_new.tgz',
                                              meta_props={client.repository.ModelMetaNames.NAME:"drug-classification",
                                                          client.repository.ModelMetaNames.TYPE:"scikit-learn_1.0",
                                                          client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
                                                         },
                                              training_data=x_train,
                                              training_target=y_train)

In [ ]:
model_id = client.repository.get_model_id(model_details)
model_id

In [ ]:
deployment = client.deployments.create(
    artifact_uid=model_id, 
    meta_props={client.deployments.ConfigurationMetaNames.NAME:"Drug Classification_deploy", 
    client.deployments.ConfigurationMetaNames.ONLINE: {}
})

In [ ]:
model_details